# Scaling Machine Learning in Python

<img src="img/saturn.png" width="400" />

## Single-node workflow

We'll first start off with a typical data preparation and machine learning workflow utilizing only the Jupyter Server.


## Monitor resource utilization

For this workshop it's important to monitor CPU and memory utilization when running various commands. It will help with understanding which operations are slow - and which ones run faster on a cluster!

To monitor resource utilization of the Jupyter Server, open a new Terminal window inside Jupyter Lab and run `htop`. You can position the window to view the notebook and terminal on the same screen:

![htop](img/htop.png)

# Load data

We'll operate with yellow taxi rides from the 2019 calendar year for this workshop. The machine learning exercises involve predicting the "tip fraction" of each ride - how much a rider will tip the driver as a fraction of the charged fare amount.

In [14]:
import s3fs

s3 = s3fs.S3FileSystem(anon=True)
files = s3.glob('s3://nyc-tlc/trip data/yellow_tripdata_2019-*.csv')

In [15]:
total_size = 0
for f in files:
    size = s3.du(f)
    total_size += size
    
    print(f"{f}, Size: {round(size / 1e6, 2)} MB")
print()
print(f"Total size: {round(total_size / 1e9, 2)} GB")

nyc-tlc/trip data/yellow_tripdata_2019-01.csv, Size: 687.09 MB
nyc-tlc/trip data/yellow_tripdata_2019-02.csv, Size: 649.88 MB
nyc-tlc/trip data/yellow_tripdata_2019-03.csv, Size: 726.2 MB
nyc-tlc/trip data/yellow_tripdata_2019-04.csv, Size: 689.21 MB
nyc-tlc/trip data/yellow_tripdata_2019-05.csv, Size: 701.54 MB
nyc-tlc/trip data/yellow_tripdata_2019-06.csv, Size: 643.49 MB
nyc-tlc/trip data/yellow_tripdata_2019-07.csv, Size: 584.39 MB
nyc-tlc/trip data/yellow_tripdata_2019-08.csv, Size: 562.39 MB
nyc-tlc/trip data/yellow_tripdata_2019-09.csv, Size: 608.97 MB
nyc-tlc/trip data/yellow_tripdata_2019-10.csv, Size: 669.17 MB
nyc-tlc/trip data/yellow_tripdata_2019-11.csv, Size: 637.81 MB
nyc-tlc/trip data/yellow_tripdata_2019-12.csv, Size: 639.11 MB

Total size: 7.8 GB


<br>

We might have enough memory on this Jupyter Server to _load_ all the files, but will run out when trying to perform later transformations and machine learning exercises. For now, we'll operate with one month.

> Note: `s3.open()` is only required because we're using an anonymous S3 connection. If you have AWS credentials set up, you could pass in just the S3 URL string.

In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter("ignore")

In [4]:
%%time

taxi = pd.read_csv(
    s3.open('s3://nyc-tlc/trip data/yellow_tripdata_2019-01.csv', mode='rb'),
    parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime']
)

CPU times: user 13.8 s, sys: 2.99 s, total: 16.8 s
Wall time: 30.2 s


In [5]:
taxi.head()

VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         1  2019-01-01 00:46:40   2019-01-01 00:53:20                1   
1         1  2019-01-01 00:59:47   2019-01-01 01:18:59                1   
2         2  2018-12-21 13:48:30   2018-12-21 13:52:40                3   
3         2  2018-11-28 15:52:25   2018-11-28 15:55:45                5   
4         2  2018-11-28 15:56:57   2018-11-28 15:58:33                5   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0            1.5           1                  N           151           239   
1            2.6           1                  N           239           246   
2            0.0           1                  N           236           236   
3            0.0           1                  N           193           193   
4            0.0           2                  N           193           193   

   payment_type  fare_amount  extra  mta_tax  tip_amount  tolls_amount  \
0             1          7.0    0.5      0.5        1.65           0.0   
1             1         14.0    0.5      0.5        1.00           0.0   
2             1          4.5    0.5      0.5        0.00           0.0   
3             2          3.5    0.5      0.5        0.00           0.0   
4             2         52.0    0.0      0.5        0.00           0.0   

   improvement_surcharge  total_amount  congestion_surcharge  
0                    0.3          9.95                   NaN  
1                    0.3         16.30                   NaN  
2                    0.3          5.80                   NaN  
3                    0.3          7.55                   NaN  
4                    0.3         55.55                   NaN

### Exercise

How many rows are in the `taxi` DataFrame?

In [8]:
<FILL IN>

SyntaxError: invalid syntax (<ipython-input-8-530f2c89953e>, line 1)

In [ ]:
len(taxi)

Memory usage is also an important consideration, as DataFrames often take more space in memory than on disk:

In [6]:
print(f"Size (MB): {taxi.memory_usage(deep=True).sum() / 1e6}")

Size (MB): 1487.551776


# Exploratory analysis

For this workshop, we will just look at column statistics. There are many more explorary analyses that can performed with `pandas` and data visualization tools

In [7]:
%%time
np.round(taxi.describe().T, 3)

CPU times: user 3.92 s, sys: 172 ms, total: 4.1 s
Wall time: 4.1 s


count     mean      std    min     25%     50%  \
VendorID               7667792.0    1.637    0.540    1.0    1.00    2.00   
passenger_count        7667792.0    1.567    1.224    0.0    1.00    1.00   
trip_distance          7667792.0    2.801    3.738    0.0    0.90    1.53   
RatecodeID             7667792.0    1.058    0.678    1.0    1.00    1.00   
PULocationID           7667792.0  165.501   66.392    1.0  130.00  162.00   
DOLocationID           7667792.0  163.753   70.364    1.0  113.00  162.00   
payment_type           7667792.0    1.292    0.473    1.0    1.00    1.00   
fare_amount            7667792.0   12.409  262.072 -362.0    6.00    8.50   
extra                  7667792.0    0.328    0.507  -60.0    0.00    0.00   
mta_tax                7667792.0    0.497    0.053   -0.5    0.50    0.50   
tip_amount             7667792.0    1.827    2.501  -63.5    0.00    1.43   
tolls_amount           7667792.0    0.317    2.024  -70.0    0.00    0.00   
improvement_surcharge  7667792.0    0.299    0.019   -0.3    0.30    0.30   
total_amount           7667792.0   15.682  262.293 -362.8    8.19   11.27   
congestion_surcharge   2811814.0    0.000    0.009    0.0    0.00    0.00   

                          75%        max  
VendorID                 2.00       4.00  
passenger_count          2.00       9.00  
trip_distance            2.80     831.80  
RatecodeID               1.00      99.00  
PULocationID           234.00     265.00  
DOLocationID           234.00     265.00  
payment_type             2.00       4.00  
fare_amount             13.50  623259.86  
extra                    0.50     535.38  
mta_tax                  0.50      60.80  
tip_amount               2.33     787.25  
tolls_amount             0.00    3288.00  
improvement_surcharge    0.30       0.60  
total_amount            16.56  623261.66  
congestion_surcharge     0.00       2.50

# Feature engineering

We are using stateless features, meaning the features values for a given observation don't depend on other observations. This is allows us to create features before performing any data splitting.

Then, split data into train/test sets.

In [8]:
# specify feature and label column names
numeric_feat = [
    'pickup_weekday', 
    'pickup_weekofyear', 
    'pickup_hour', 
    'pickup_week_hour', 
    'pickup_minute', 
    'passenger_count',
]
categorical_feat = [
    'PULocationID', 
    'DOLocationID',
]
features = numeric_feat + categorical_feat
y_col = 'tip_fraction'

In [40]:
def prep_df(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Generate features from a raw taxi dataframe.
    '''
    df = df[df.fare_amount > 0]  # avoid divide-by-zero
    df['tip_fraction'] = df.tip_amount / df.fare_amount
     
    df['pickup_weekday'] = df.tpep_pickup_datetime.dt.weekday
    df['pickup_weekofyear'] = df.tpep_pickup_datetime.dt.weekofyear
    df['pickup_hour'] = df.tpep_pickup_datetime.dt.hour
    df['pickup_week_hour'] = (df.pickup_weekday * 24) + df.pickup_hour
    df['pickup_minute'] = df.tpep_pickup_datetime.dt.minute
    df = df[features + [y_col]].astype(float).fillna(-1)
    
    return df

### Exercise

Create a new DataFrame called `taxi_feat` by running the `prep_df()` function against the `taxi` DataFrame, then preview the data.

In [ ]:
<FILL IN>

In [41]:
taxi_feat = prep_df(taxi)
taxi_feat.head()

pickup_weekday  pickup_weekofyear  pickup_hour  pickup_week_hour  \
0             1.0                1.0          0.0              24.0   
1             1.0                1.0          0.0              24.0   
2             4.0               51.0         13.0             109.0   
3             2.0               48.0         15.0              63.0   
4             2.0               48.0         15.0              63.0   

   pickup_minute  passenger_count  PULocationID  DOLocationID  tip_fraction  
0           46.0              1.0         151.0         239.0      0.235714  
1           59.0              1.0         239.0         246.0      0.071429  
2           48.0              3.0         236.0         236.0      0.000000  
3           52.0              5.0         193.0         193.0      0.000000  
4           56.0              5.0         193.0         193.0      0.000000

Split into train/test sets

In [42]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    taxi_feat[features], 
    taxi_feat[y_col], 
    test_size=0.3,
    random_state=42
)

In [43]:
X_train.shape, y_train.shape

((5360764, 8), (5360764,))

In [44]:
X_test.shape, y_test.shape

((2297471, 8), (2297471,))

# Train model

We'll train a linear model to predict `tip_fraction`. 

While `PULocationID` and `DOLocationID` are numeric, they do not have contain any ordinal information and need to be treated as categoricals. We also need to scale the true numeric features.

Evaluate the model against the test set using RMSE.

In [45]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error

pipeline = Pipeline(steps=[
    ('preprocess', ColumnTransformer(transformers=[
        ('num', StandardScaler(), numeric_feat),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_feat),
    ])),
    ('clf', ElasticNet(normalize=False, max_iter=100)),
])

Make sure to have `htop` open while running the below cells, you'll want to watch the CPU and memory utilization.

In [46]:
%%time
_ = pipeline.fit(X_train, y_train)

CPU times: user 11.3 s, sys: 983 ms, total: 12.3 s
Wall time: 12.1 s


In [47]:
%%time
preds = pipeline.predict(X_test)

CPU times: user 1.44 s, sys: 305 ms, total: 1.75 s
Wall time: 1.55 s


In [48]:
mean_squared_error(y_test, preds, squared=False)

5.118045215591297

## Hooray!

We trained a terrible model. But that's okay. The point of this workshop is to scale our work, not make the model perfect!

# Let's step things up

We were able to train a single, simple model on a sample of the taxi data (single month from 2019). In most model building settings more data and more compute would be required. We will explore the following scenarios where the single-node environment has challenges - and see how Dask solves these problems!

1. Load and process large dataset (full year's data)
1. Train model with large dataset
1. Tune hyperparameters (train numerous models)
1. Predict over large dataset

# 1. Load and process large dataset

Possible, takes too long

In [ ]:
# for loop pandas

# 2. Train model with large dataset

Blows up

In [ ]:
# pipeline.fit()

![explosion](img/explosion.gif)

Oh no! Looks like we don't have enough memory to train using this pipeline and data size. Let's try with a smaller sample
> Note: Since the kernel died, you will need to re-run the above cells, except for the `pipeline.fit()` one of course!

# 3. Tune hyperparameters

Takes too long

In [49]:
# grid search

# 4. Predict over large dataset

Takes too long

In [ ]:
# pipeline.predict()